In [ ]:
import cv2
import numpy as np

# load the image and scale to 0..1
image = cv2.imread('lena512.bmp', cv2.IMREAD_GRAYSCALE).astype(float) / 255.0

# load + show the original
cv2.imshow('original', image)

# horizontal edge detector
kernel = np.array([[1, 0, -1],
                   [1, 0, -1],
                   [1, 0, -1]])
filtered = cv2.filter2D(src=image, kernel=kernel, ddepth=-1)
cv2.imshow('horizontal edges', filtered)

# vertical edge detector
kernel = np.array([[1,  1,  1],
                   [0,  0,  0],
                   [-1, -1, -1]])
filtered = cv2.filter2D(src=image, kernel=kernel, ddepth=-1)
cv2.imshow('vertical edges', filtered)

# blurring ("box blur", because it's a box of ones)
kernel = np.array([[1, 1, 1],
                   [1, 1, 1],
                   [1, 1, 1]]) / 9.0
filtered = cv2.filter2D(src=image, kernel=kernel, ddepth=-1)
cv2.imshow('blurred', filtered)

# sharpening
kernel = (np.array([[-1, -1, -1],
                    [-1,  9, -1],
                    [-1, -1, -1]]))
filtered = cv2.filter2D(src=image, kernel=kernel, ddepth=-1)
cv2.imshow('sharpened', filtered)

# wait and quit
cv2.waitKey(0)
cv2.destroyAllWindows()

# Manual convol

In [ ]:
import cv2
import numpy as np

# load the image and scale to 0..1
image = cv2.imread('lena512.bmp', cv2.IMREAD_GRAYSCALE).astype(float) / 255.0

# do-it-yourself convolution:
# For each pixel in the input image, we'll inspect its neighborhood. A 3x3 kernel thus peeks
# at every neighbor of a specific pixel (there are 8 pixel neighbors) whereas a 5x5 kernel
# peeks at two pixels in every direction (i.e. 24 pixel neighbors).

# A kernel of all ones is called a box blur and is simply averaging all neighbors (sum all, optionally divide by count).
kernel = (np.array([[1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1]]))

# the weighed pixels have to be in range 0..1, so we divide by the sum of all kernel
# values afterwards
kernel_sum = kernel.sum()

# fetch the dimensions for iteration over the pixels and weights
i_width, i_height = image.shape[0], image.shape[1]
k_width, k_height = kernel.shape[0], kernel.shape[1]

# prepare the output array
filtered = np.zeros_like(image)

# Iterate over each (x, y) pixel in the image ...
for y in range(i_height):
    for x in range(i_width):
        weighted_pixel_sum = 0

        # Iterate over each weight at (kx, ky) in the kernel defined above ...
        # We interpret the kernel weights in a way that the 'central' weight is at (0, 0);
        # so the coordinates in the kernel are:
        #
        #  [ (-1,-1),  (0,-1),  (1,-1)
        #    (-1, 0),  (0, 0),  (1, 0)
        #    (-1, 1),  (0, 1),  (1, 1)
        #
        # This way, the pixel at image[y,x] is multiplied with the kernel[0,0]; analogous,
        # image[y-1,x] is multiplied with kernel[-1,0] etc.
        # The filtered pixel is then the sum of these, so that
        #
        #   weighted_pixel_sum = image[y-1,x-1] * kernel[-1,-1] +
        #                        image[y-1,x  ] * kernel[-1, 0] +
        #                        image[y-1,x+1] * kernel[-1, 1] +
        #                        image[y,  x-1] * kernel[ 0, 1] +
        #                        image[y,  x  ] * kernel[ 0, 0] +
        #                        etc.

        for ky in range(-(k_height // 2), k_height - 1):
            for kx in range(-(k_width // 2), k_width - 1):
                pixel = 0
                pixel_y = y - ky
                pixel_x = x - kx

                # boundary check: all values outside the image are treated as zero.
                # This is a definition and implementation dependent, it's not a property of the convolution itself.
                if (pixel_y >= 0) and (pixel_y < i_height) and (pixel_x >= 0) and (pixel_x < i_width):
                    pixel = image[pixel_y, pixel_x]

                # get the weight at the current kernel position
                # (also un-shift the kernel coordinates into the valid range for the array.)
                weight = kernel[ky + (k_height // 2), kx + (k_width // 2)]

                # weigh the pixel value and sum
                weighted_pixel_sum += pixel * weight

        # finally, the pixel at location (x,y) is the sum of the weighed neighborhood
        filtered[y, x] = weighted_pixel_sum / kernel_sum

cv2.imshow('DIY convolution', filtered)

# wait and quit
cv2.waitKey(0)
cv2.destroyAllWindows()

# Manual Concolucion en Español

In [ ]:
import cv2
import numpy as np
import csv
import pandas as pd 


# carga la imagen en escala de 0..1
image = cv2.imread('lena512.bmp', cv2.IMREAD_GRAYSCALE).astype(float) / 255.0

# convolución 
# Para cada píxel en la imagen de entrada, inspeccionaremos su vecino. Un núcleo 3x3  se asoma así
# en cada vecino de un píxel específico (hay vecinos de 8 píxeles) mientras que un núcleo de 5x5
# mira a dos píxeles en todas las direcciones (es decir, 24 píxeles vecinos).


# Un núcleo 3x3 de suavisado y que promedia todos los vecinos (suma todos, opcionalmente divide por conteo).
kernel_1 = (np.array([[1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1]]))

# detector de borde vertical
kernel = np.array([[1,  1,  1],
                   [0,  0,  0],
                   [-1, -1, -1]])

# detector de borde horizontal
kernel_3 = np.array([[1, 0, -1],
                   [1, 0, -1],
                   [1, 0, -1]])

# filtro  5x5 de suavizado gaussiano
kernel_g = (np.array([[1, 4, 6, 4, 1],
                [4, 16, 24, 16, 4],
                [6, 24, 36, 24, 6],
                [4, 16, 24, 16, 4],
                [1, 4, 6, 4, 1]])) /256.0

# filtro 5x5 detector de bordes,
kernel_b = (np.array([[1, 2, 0, -2, -1],
                [1, 2, 0, -2, -1],
                [1, 2, 0, -2, -1],
                [1, 2, 0, -2, -1],
                [1, 2, 0, -2, -1]])) 


# los píxeles deben estar en el rango 0..1, por lo que dividimos por la suma de todos los núcleos
kernel_sum = kernel.sum()

if kernel_sum == 0: kernel_sum = 1

print('Kernel sum procesando ', str(kernel_sum))
# busca las dimensiones para la iteración sobre los píxeles
i_width, i_height = image.shape[0], image.shape[1]
k_width, k_height = kernel.shape[0], kernel.shape[1]

# preparar la matriz de salida
filtered = np.zeros_like(image)

# Itera sobre cada (x, y) píxel en la imagen ...
for y in range(i_height):
    for x in range(i_width):
        weighted_pixel_sum = 0

        # Iterar sobre cada valor en (kx, ky) en el núcleo definido anteriormente ...
        # Interpretamos los valores del núcleo de manera que el valor 'central' esté en (0, 0);
        # entonces las coordenadas en el núcleo son:
        #
        #  [ (-1,-1),  (0,-1),  (1,-1)
        #    (-1, 0),  (0, 0),  (1, 0)
        #    (-1, 1),  (0, 1),  (1, 1)
        #
        # De esta manera, el píxel en la imagen [y, x] se multiplica con el núcleo [0,0]; análogo,
        # imagen [y-1, x] se multiplica con el núcleo [-1,0] etc.
        # El píxel filtrado es la suma de estos, de modo que
        #
        #   weighted_pixel_sum = image[y-1,x-1] * kernel[-1,-1] +
        #                        image[y-1,x  ] * kernel[-1, 0] +
        #                        image[y-1,x+1] * kernel[-1, 1] +
        #                        image[y,  x-1] * kernel[ 0, 1] +
        #                        image[y,  x  ] * kernel[ 0, 0] +
        #                        etc.

        for ky in range(-(k_height // 2), k_height - 1):
            for kx in range(-(k_width // 2), k_width - 1):
                pixel = 0
                pixel_y = y - ky
                pixel_x = x - kx
 
               # verifica límites: todos los valores fuera de la imagen se tratan como cero.
               # Esta es una definición y una implementación dependientes, no es una propiedad de la convolución en sí.
                if (pixel_y >= 0) and (pixel_y < i_height) and (pixel_x >= 0) and (pixel_x < i_width):
                    pixel = image[pixel_y, pixel_x]

                #  obtener el ancho en la posición actual del kernel
                # (también desplaza las coordenadas del núcleo al rango válido para la matriz).
                weight = kernel[ky + (k_height // 2)-1, kx + (k_width // 2)-1]

                # valor de píxeles y la suma
                weighted_pixel_sum += pixel * weight
                
        # finalmente, el píxel en la ubicación (x, y) es la suma del vecino 
        filtered[y, x] = weighted_pixel_sum / kernel_sum
    
cv2.imshow('Salida convolucion', filtered)

pd.DataFrame(filtered).to_csv("file.csv") 
# wait and quit
cv2.waitKey(0)
cv2.destroyAllWindows()

Kernel sum procesando  1
